# Наивный Байес и спам

В этом задании вам нужно будет реализовать классификацию методом Байеса в предположении независимости компонент `x`.

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from collections import Counter
from nltk.stem.snowball import SnowballStemmer

In [2]:
import re
def read_dataset(filename):
    file = open(filename, encoding="utf-8")
    x = []
    y = []
    for line in file:
        cl, sms = re.split("^(ham|spam)[\t\s]+(.*)$", line)[1:3]
        x.append(sms)
        y.append(cl)
    return np.array(x, dtype=np.str), np.array(y, dtype=np.str)

In [3]:
def get_precision_recall_accuracy(y_pred, y_true):
    classes = np.unique(list(y_pred) + list(y_true))
    true_positive = dict((c, 0) for c in classes)
    true_negative = dict((c, 0) for c in classes)
    false_positive = dict((c, 0) for c in classes)
    false_negative = dict((c, 0) for c in classes)
    for c_pred, c_true in zip(y_pred, y_true):
        for c in classes:
            if c_true == c:
                if c_pred == c_true:
                    true_positive[c] = true_positive.get(c, 0) + 1
                else:
                    false_negative[c] = false_negative.get(c, 0) + 1
            else:
                if c_pred == c:
                    false_positive[c] = false_positive.get(c, 0) + 1
                else:
                    true_negative[c] = true_negative.get(c, 0) + 1
    precision = dict((c, 1 if true_positive[c] + false_positive[c] == 0 else true_positive[c] / (true_positive[c] + false_positive[c])) for c in classes)
    recall = dict((c, 1 if true_positive[c] + false_negative[c] == 0 else  true_positive[c] / (true_positive[c] + false_negative[c])) for c in classes)
    accuracy = sum([true_positive[c] for c in classes]) / len(y_pred)
    return precision, recall, accuracy

In [4]:
def read_csv_dataset(filename):
    df = pandas.read_csv(filename, encoding='latin-1')
    df.dropna(how="any", inplace=True, axis=1)
    df = df.rename(columns={'v1': 'y', 'v2': 'x'})
    return df.x, df.y

In [5]:
X, y = read_csv_dataset("spam.csv")

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9)

#### Naive Bayes

Реализуйте классификацию методом Байеса. При реализации следует считать, что значения каждой компоненты дискретны, а `p(x|y)` имеет биномиальное распределение.

#### Методы
`fit(X, y)` - оценивает параметры распределения `p(x|y)` для каждого `y`. 

`log_proba(X)` - для каждого элемента набора `X` считает логарифм вероятности отнести его к каждому классу. По теореме Байеса: `p(y|x) = p(y)p(x|y)/p(x)`. Соответственно, после логарифмирования: `log p(y|x) = log p(y) + log p(x|y) - log p(x)`. Поскольку `log p(x)` является независящей от класса `y` константой, это слагаемое никак не влияет на решение задачи максимизации. Поэтому его можно просто убрать для простоты.

In [7]:
class NaiveBayes:
    def __init__(self, alpha):
        self.alpha = alpha # Параметр аддитивной регуляризации
    
    def fit(self, X, y):
        y = np.array(y)
        
        self.classes, counts = np.unique(y, return_counts=True)
        self.log_classes_proba = np.log(counts / len(y))

        n_features = X.shape[1]
        self.log_aposterior_proba = np.zeros(shape=(len(self.classes), n_features))
        
        for class_id, class_name in enumerate(self.classes):
            for feature_id in range(n_features):
                self.log_aposterior_proba[class_id, feature_id] = \
                        np.log((np.sum(X[np.argwhere(y == class_name), feature_id]) + self.alpha) / (np.sum(X[:, feature_id]) + self.alpha * len(self.classes)))
                                             
    def predict(self, X):
        return [self.classes[i] for i in np.argmax(self.log_proba(X), axis=1)]
    
    def log_proba(self, X):
        return self.log_classes_proba + X @ self.log_aposterior_proba.T

#### Bag of words
Теперь реализуем метод bag of words. Задача состоит в том, чтобы посчитать количество вхождений каждого слова в предложение.

#### Методы
`__init__(X, voc_limit)` - инициализирует BoW, составляя словарь, который будет использоваться для векторизации предложений. Размер словаря должен быть не больше `voc_limit`, в него должны входить самые часто встречающиеся в выборке слова.

`transform(X)` - векторизует сообщения из набора.

In [8]:
class BoW:
    def __init__(self, X, voc_limit=1000):
        X = np.array(X)
        counter = Counter()
        
        for x in X:
            counter.update(x.split(' '))
        
        vocab, s = zip(*counter.most_common(voc_limit))
        self.vocabulary_ = dict(zip(vocab, np.arange(voc_limit)))
        
    def transform(self, X):
        transformed_X = np.zeros((X.shape[0], len(self.vocabulary_)), dtype=int)
        for line_ind, x in enumerate(X):
            words = x.split(' ')
            for word in words:
                index = self.vocabulary_.get(word)
                if index is not None:
                    transformed_X[line_ind][index] += 1
                    
        return transformed_X    

In [9]:
bow = BoW(X_train, voc_limit=500)
X_train_bow = bow.transform(X_train)
X_test_bow = bow.transform(X_test)

In [10]:
predictor = NaiveBayes(0.001)
predictor.fit(X_train_bow, y_train)
precision, recall, accuracy = get_precision_recall_accuracy(predictor.predict(X_test_bow), y_test)
print(f'precision = {precision}', 
      f'recall = {recall}',
      f'accuracy = {accuracy}', sep='\n')

precision = {'ham': 0.9035250463821892, 'spam': 0.9473684210526315}
recall = {'ham': 0.9979508196721312, 'spam': 0.2571428571428571}
accuracy = 0.9050179211469535


#### Стемминг
Теперь добавим в BoW стемминг. Для этого нам понадобится класс SnowballStemmer из пакета `nltk`

#### Методы
`__init__(X, voc_limit)` - инициализирует BoW, составляя словарь, который будет использоваться для векторизации предложений. Размер словаря должен быть не больше `voc_limit`, в него должны входить самые часто встречающиеся в выборке слова.

`transform(X)` - векторизует сообщения из набора.

In [11]:
class BowStem:
    def __init__(self, X, voc_limit=1000):
        X = np.array(X)
        counter = Counter()
        self.stemmer = SnowballStemmer("english")
        
        for x in X:
            counter.update([self.stemmer.stem(word) for word in x.split(' ')])
        
        vocab, s = zip(*counter.most_common(voc_limit))
        self.vocabulary_ = dict(zip(vocab, np.arange(voc_limit)))
        
    def transform(self, X):
        transformed_X = np.zeros((X.shape[0], len(self.vocabulary_)), dtype=int)
        for line_ind, x in enumerate(X):
            words = [self.stemmer.stem(word) for word in x.split(' ')]
            for word in words:
                index = self.vocabulary_.get(word)
                if index is not None:
                    transformed_X[line_ind][index] += 1
                    
        return transformed_X 

In [12]:
bows = BowStem(X_train, voc_limit=500)
X_train_bows = bows.transform(X_train)
X_test_bows = bows.transform(X_test)

In [13]:
predictor = NaiveBayes(0.001)
predictor.fit(X_train_bows, y_train)
get_precision_recall_accuracy(predictor.predict(X_test_bows), y_test)

({'ham': 0.9155722326454033, 'spam': 1.0},
 {'ham': 1.0, 'spam': 0.35714285714285715},
 0.9193548387096774)